# PySpark and SparkR Arrow optimization examples

First, enable R cell magic as below.

In [1]:
import rpy2.rinterface
%load_ext rpy2.ipython

After that, locate SparkR library directroy for SparkR.

In [2]:
%%R
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))

R[write to console]: 
Attaching package: ‘SparkR’


R[write to console]: The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window


R[write to console]: The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union




Initializes a Spark session for PySaprk as well.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Exception: Java gateway process exited before sending its port number

## R DataFrame <> Spark DataFrame with Arrow optimization

In [ ]:
%%R
sparkR.session(master = "local[*]", sparkConfig = list(spark.sql.execution.arrow.sparkr.enabled = "true"))

In [ ]:
%%R
collect(createDataFrame(iris))

## `dapply` and `gapply` with Arrow optimization

In [ ]:
%%R
sparkR.session(master = "local[*]", sparkConfig = list(spark.sql.execution.arrow.sparkr.enabled = "true"))

In [ ]:
%%R
df <- createDataFrame(mtcars)
collect(gapply(df,
               "gear",
               function(key, group) {
                 data.frame(gear = key[[1]], disp = mean(group$disp) > group$disp)
               },
               structType("gear double, disp boolean")))

In [ ]:
%%R
df <- createDataFrame(mtcars)
collect(dapply(df, function(rdf) { data.frame(rdf$gear + 1) }, structType("gear double")))

## Pandas DataFrame <> Spark DataFrame with Arrow optimization

In [ ]:
import pandas as pd

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.createDataFrame(pd.DataFrame({'a': range(10)})).toPandas()